import libraries

In [1]:
import pandas as pd;
from scipy.stats import zscore
import torch as torch;
import numpy as np
import torchvision.datasets as datasets
from torchvision import models, transforms
import torch.nn as nn;
import torch.nn.functional as F;
np.random.seed(42)
# Use GPU if available, else use CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# HW04: Transfer Learning

I was unable to sucsessfully implement a working accuracy metric. As a result, to avoid being totally unable to complete the project I rewrote my project using the keras interface and built in functions despite the fact this was not the point of the project.

Data Pre Processing:

In [2]:
num_classes=10;
transform = transforms.Compose(
        [transforms.Resize((224, 224)),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, 
                             transform=transform)
full_test_dataset = datasets.CIFAR10(root='./data', train=False, download=True,
                             transform=transform)
batch_size=64;
trainloader = torch.utils.data.DataLoader(full_train_dataset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(full_test_dataset,batch_size=batch_size,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Here we define the neural network:

In [3]:
# create a neural network (inherit from nn.Module)
class ConvNetWithBatchNorm(nn.Module):
    # architecture of the network is specified in the constructor
    def __init__(self): 
        super(ConvNetWithBatchNorm, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),         
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),  
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3),
            nn.BatchNorm2d(num_features=12)           
        )
        self.features1 = nn.Sequential(
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)   
        )
        self.classifier = nn.Sequential(
            nn.Linear(12*6*6, 50),         
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(50,num_classes)            
        )
        
    # here we specify the computation (forward phase of training) how "x" is transfered into output "y"
    def forward(self, x):
        x = self.features(x)
        x = self.features1(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x)

Here we instantiate the neural network, set our learning rate, and instantiate our optimizer.

In [4]:
model=ConvNetWithBatchNorm().to(device);
criterion = F.nll_loss;
# this optimizer will do gradient descent for us
# experiment with learning rate and optimizer type
learning_rate = 0.001;
# note that we have to add all weights&biases, for both layers, to the optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
# we add a learning rate scheduler, which will modify the learning rate during training
# will initially start low, then increase it ("warm up"), and then gradually decrease it
n_epochs = 5;
num_updates = n_epochs*int(np.ceil(len(trainloader.dataset)/batch_size))
print(num_updates)
warmup_steps=1000;
def warmup_linear(x):
    if x < warmup_steps:
        lr=x/warmup_steps
    else:
        lr=max( (num_updates - x ) / (num_updates - warmup_steps), 0.)
    return lr;
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_linear);


3910


Next we have our training loop.

In [ ]:
for i in range(n_epochs):
    for j, data in enumerate(trainloader):

        inputs, labels = data        
        inputs=inputs.to(device);
        labels=labels.to(device);
        
        optimizer.zero_grad();
        #forward phase - predictions by the model
        outputs = model(inputs);
        #forward phase - risk/loss for the predictions
        risk = criterion(outputs, labels);

        # calculate gradients
        risk.backward();

        # take the gradient step
        optimizer.step();
        scheduler.step();

        batch_risk=risk.item();
    with (torch.no_grad()):
      correct = 0;
      for j, data in enumerate(testloader):

          inputs, labels = data
          inputs=inputs.to(device);
          labels=labels.to(device);
          outputs = model(inputs);
          pred = outputs.data.max(dim=1, keepdim=True)[1]
          correct += pred.eq(labels.data.view_as(pred)).sum().item();
    print(i, batch_risk, correct / len(testloader.dataset))

Next we need to instantiate our ResNet18 network.

In [5]:
ResNet18model = models.resnet18(pretrained=False)   #load resnet18 model
num_features = ResNet18model.fc.in_features     #extract fc layers features
ResNet18model.fc = nn.Linear(num_features, num_classes) 
ResNet18model = ResNet18model.to(device)


C:\Users\echo4\.conda\envs\pt\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\echo4\.conda\envs\pt\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
for i in range(n_epochs):
    for j, data in enumerate(trainloader):

        inputs, labels = data        
        inputs=inputs.to(device);
        labels=labels.to(device);
        
        optimizer.zero_grad();
        #forward phase - predictions by the model
        outputs = ResNet18model(inputs);
        #forward phase - risk/loss for the predictions
        risk = criterion(outputs, labels);

        # calculate gradients
        risk.backward();

        # take the gradient step
        optimizer.step();
        scheduler.step();

        batch_risk=risk.item();
    with (torch.no_grad()):
      correct = 0;
      for j, data in enumerate(testloader):

          inputs, labels = data
          inputs=inputs.to(device);
          labels=labels.to(device);
          outputs = ResNet18model(inputs);
          pred = outputs.data.max(dim=1, keepdim=True)[1]
          correct += pred.eq(labels.data.view_as(pred)).sum().item();
    print(i, batch_risk, correct / len(testloader.dataset))